<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/%D0%93%D0%BB%D0%B0%D0%B2%D0%B0_6_3_%D0%9A%D0%BE%D0%BC%D0%BF%D0%BB%D0%B5%D0%BA%D1%81%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%B2%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D1%8B%D1%85_%D1%80%D1%8F%D0%B4%D0%BE%D0%B2_%D0%B8_%D0%BF%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BD%D0%B0_%D1%80%D0%B5%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D1%85_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D1%81_%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%D0%BC_AutoGluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Комплексный анализ временных рядов и прогнозирование на реальных данных с использованием AutoGluon**

**Тема:** Построение end-to-end решения для прогнозирования цен на недвижимость, включая предобработку данных, feature engineering, обучение модели и глубокий анализ результатов.

**Цель:** Продемонстрировать полный цикл работы Data Scientist на практике, от сырых данных до готовых выводов для научной статьи.



### **Структура лекции (соответствует разделам кода):**

1.  **Введение и подготовка среды**
2.  **Предварительный анализ и предобработка данных (Data Preprocessing)**
3.  **Разведочный анализ данных (EDA - Exploratory Data Analysis)**
4.  **Статистический анализ и проверка гипотез**
5.  **Feature Engineering и подготовка данных для модели**
6.  **Прогнозирование с помощью AutoGluon**
7.  **Диагностика модели и анализ остатков**
8.  **Интерпретация модели и финальные результаты**



### **#1. Введение и подготовка среды**

```python
# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
...
```

*   **Импорт библиотек:** Код начинается с импорта всех необходимых библиотек. Это стандартная практика.
    *   `pandas`, `numpy`: для работы с данными.
    *   `matplotlib`, `seaborn`, `plotly`: для визуализации. Использование `plotly` говорит о стремлении к интерактивным и публикационным графикам.
    *   `scipy.stats`, `statsmodels`: для статистического анализа и анализа временных рядов (тесты, ACF/PACF, декомпозиция).
    *   `sklearn`: для метрик и кросс-валидации.
    *   `autogluon.tabular`: основной инструмент для автоматического машинного обучения. Он автоматически подберет и обучит ансамбль моделей (GBM, CAT, XGB и т.д.).
    *   `shap`: для интерпретируемости модели и анализа важности признаков.
    *   `tqdm`: для отображения прогресса.
*   **Настройки отображения:** Увеличение максимального числа колонок и ширины вывода для `pandas`, установка стилей графиков. Это важно для удобства анализа.
*   **Google Colab:** Код предназначен для запуска в среде Google Colab, о чем говорит монтаж Google Drive. Это популярный выбор для экспериментов из-за бесплатного доступа к GPU/TPU.

**Ключевой вывод:** Подготовлена полноценная среда для анализа, покрывающая все этапы: от обработки данных до продвинутого ML и визуализации.



### **#2. Предварительный анализ и предобработка данных**

**Цель:** Превратить сырые данные в пригодный для анализа и моделирования набор.

```python
# Загрузка данных
df = pd.read_csv('/content/drive/MyDrive/Datasets/tatarstan_dataset.csv')
```
*   **Загрузка данных:** Данные загружаются из CSV-файла, предположительно содержащего информацию о продажах недвижимости в Татарстане.

```python
print("Размер датасета:", df.shape)
df.info()
df.describe().round(2)
missing_df = pd.DataFrame({'Количество': missing_data, 'Процент': missing_percent})
```
*   **Первичный анализ:** Сразу же оценивается размер данных, типы столбцов, наличие пропусков. `df.describe()` дает представление о распределении числовых признаков. Это первый и обязательный шаг.

```python
# Обработка пропущенных значений
def handle_missing_values(df):
    # Числовые признаки - медиана
    # Категориальные признаки - мода
```
*   **Обработка пропусков:** Пропуски в числовых признаках заполняются **медианой** (устойчива к выбросам), в категориальных — **модой** (наиболее частым значением). Это стандартные и надежные стратегии.

```python
# Создание datetime колонки
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df = df.sort_values('datetime').reset_index(drop=True)
```
*   **Работа с временем:** Создание единого столбца `datetime` и сортировка по нему — **критически важный шаг для временных рядов**. Без этого последующий анализ лагов и скользящих статистик будет некорректным.

```python
# Анализ и обработка выбросов
def detect_outliers(df, column, threshold=3):
    z_scores = np.abs(stats.zscore(df[column]))
...
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
```
*   **Борьба с выбросами:** Применяются два метода.
    1.  **Z-Score** (`detect_outliers`): для детектирования и анализа выбросов.
    2.  **IQR (Interquartile Range)** (`remove_outliers_iqr`): для их непосредственного удаления. Метод IQR считается более робастным, чем z-score, особенно для ненормальных распределений. Удаляются значения, выходящие за пределы `[Q1 - 1.5*IQR, Q3 + 1.5*IQR]`.

**Ключевой вывод:** Данные были тщательно очищены от пропусков и выбросов, а также приведены к формату, пригодному для анализа временных рядов.



### **#3. Разведочный анализ данных (EDA)**

**Цель:** Понимание данных, выявление закономерностей, аномалий и взаимосвязей.

```python
# Распределение целевой переменной
sns.histplot(df_clean['price'], kde=True, bins=50)
sns.histplot(np.log1p(df_clean['price']), kde=True, bins=50)
```
*   **Анализ целевой переменной (Price):** Исходное распределение цен скорее всего имеет **длинный хвост (right-skewed)**. Логарифмирование (`np.log1p`) помогает приблизить распределение к нормальному, что часто полезно для линейных моделей.

```python
# Временной ряд средней цены
daily_prices = df_clean.groupby('date')['price'].agg(['mean', 'median', 'count']).reset_index()
fig = make_subplots(rows=2, cols=1, subplot_titles=('Средняя цена по дням', 'Количество объявлений по дням'))
```
*   **Анализ временных рядов:** Строятся два ключевых графика: динамика средней цены и динамика количества объявлений. Важно искать **тренды, сезонность и выбросы**. Например, падение количества объявлений может объяснять всплески волатильности цен.

```python
# Сезонность и тренды
decomposition = seasonal_decompose(ts_series, period=30, model='additive')
```
*   **Декомпозиция временного ряда:** Классический метод разделения ряда на составляющие:
    *   **Тренд (Trend):** Долгосрочная направленность.
    *   **Сезонность (Seasonality):** Периодические колебания (например, ежеквартальные).
    *   **Остаток (Resid):** Случайный шум, который не объясняется моделью.

```python
# Анализ корреляций
correlation_matrix = df_clean[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
```
*   **Матрица корреляций:** Позволяет быстро увидеть линейные зависимости между числовыми признаками. Например, ожидаема высокая корреляция между `area` и `price`.

```python
# Анализ категориальных переменных
categorical_cols = ['region', 'building_type', 'object_type']
df_clean[col].value_counts().plot(kind='bar')
```
*   **Анализ категориальных признаков:** Столбчатые диаграммы показывают распределение объектов по регионам, типам зданий и т.д. Помогает выявить дисбаланс классов.

```python
# Географическое распределение цен
plt.scatter(df_clean['geo_lon'], df_clean['geo_lat'], c=df_clean['price'], cmap='viridis')
```
*   **Гео-визуализация:** Точечная карта, где цвет точки означает цену. Позволяет визуально идентифицировать дорогие и дешевые районы (например, центр города vs окраины).

**Ключевой вывод:** EDA выявило особенности данных: ненормальное распределение цены, наличие тренда и сезонности, географическую зависимость цены и корреляции между признаками.



### **#4. Статистический анализ и проверка гипотез**

**Цель:** Подтвердить или опровергнуть качественные предположения статистическими методами.

```python
# Тест на стационарность (ADF и KPSS)
def test_stationarity(timeseries):
    dftest = adfuller(timeseries, autolag='AIC')
    kpsstest = kpss(timeseries, regression='c')
```
*   **Тесты на стационарность:** Крайне важны для прогнозирования.
    *   **ADF (Augmented Dickey-Fuller):** Нулевая гипотеза — ряд нестационарен. Малое p-value (<0.05) позволяет отклонить H0 и считать ряд стационарным.
    *   **KPSS (Kwiatkowski-Phillips-Schmidt-Shin):** Нулевая гипотеза — ряд стационарен. Малое p-value говорит *против* стационарности.
    *   Обычно ряд требует преобразований (дифференцирования) если ADF говорит о нестационарности, а KPSS это подтверждает.

```python
# Автокорреляция и частичная автокорреляция
plot_acf(ts_series.dropna(), ax=ax1, lags=40)
plot_pacf(ts_series.dropna(), ax=ax2, lags=40)
```
*   **ACF/PACF:** Графики автокорреляции помогают определить параметры для классических моделей временных рядов (ARIMA). ACF показывает общую корреляцию с лагами, PACF — только "чистую" корреляцию с конкретным лагом.

```python
# Проверка статистических гипотез
f_stat, p_value = stats.f_oneway(*region_prices)
```
*   **ANOVA (Analysis of Variance):** Проверяет гипотезу о том, что средние значения цен **статистически значимо различаются** между разными группами (регионами, типами зданий). Малое p-value (<0.05) подтверждает, что признак действительно важен для объяснения различий в цене.

**Ключевой вывод:** Статистические тесты подтвердили нестационарность ряда и статистическую значимость различий цен в зависимости от категориальных признаков.



### **#5. Feature Engineering и подготовка данных**

**Цель:** Создать новые информативные признаки, которые помогут модели лучше уловить закономерности.

```python
# Преобразование Бокса-Кокса
def apply_boxcox(series):
    transformed, lmbda = stats.boxcox(series_positive)
```
*   **Преобразование Бокса-Кокса:** Более продвинутый способ, чем логарифмирование, для нормализации распределения и стабилизации дисперсии. Подбирает оптимальный параметр `lambda` автоматически.

```python
# Создание временных признаков
def create_time_features(df):
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    ...
    df['season'] = df['month'] % 12 // 3 + 1
```
*   **Временные признаки:** Из datetime извлекаются составляющие (год, месяц, день недели, время года и т.д.). Это помогает модели выучить **сезонные паттерны** (например, цены выше весной, ниже осенью).

```python
# Создание лаговых и скользящих статистик
def create_lag_features(df, group_col='region', target_col='price', lags=[1, 7, 30]):
    df[f'{target_col}_lag_{lag}'] = df.groupby(group_col)[target_col].shift(lag)
    df[f'{target_col}_rolling_mean_{window}'] = ...rolling(window...).mean()
```
*   **Лаги и скользящие окна:** Это **ключевой прием** для прогнозирования временных рядов табличными методами.
    *   **Лаг (Lag):** Цена 1, 7 или 30 дней назад. Модель видит, что было в прошлом.
    *   **Скользящее среднее (Rolling Mean):** Сглаживает ряд, показывает тренд.
    *   **Скользящее стандартное отклонение (Rolling Std):** Показывает волатильность.
    *   Группировка по `region` перед сдвигом — блестящая идея! Она создает эти признаки в разрезе каждого региона, что гораздо осмысленнее, чем по всему датасету.

```python
# Кодирование категориальных переменных
le_region = LabelEncoder()
df['region_encoded'] = le_region.fit_transform(df['region'])
df = pd.get_dummies(df, columns=categorical_cols, prefix=categorical_cols)
```
*   **Кодирование признаков:**
    *   **Label Encoding** для региона, так как это порядковый признак с множеством категорий.
    *   **One-Hot Encoding** для других категориальных признаков, так как у них мало уникальных значений.

**Ключевой вывод:** Создан богатый набор признаков, включая временные, лаговые и статистические, которые являются залогом успешного прогноза.



### **#6. Прогнозирование с AutoGluon**

**Цель:** Использовать мощь AutoML для автоматического построения и оптимизации ансамбля моделей.

```python
# Разделение на train/test с учетом временного порядка
split_date = df_final['datetime'].quantile(0.8)
train_data = df_final[df_final['datetime'] < split_date]
test_data = df_final[df_final['datetime'] >= split_date]
```
*   **Временное разделение:** Важно! Никогда не нужно делать случайное разделение для временных рядов. Данные должны быть разделены по времени: старые данные для обучения, новые — для тестирования. Это симулирует реальный процесс прогнозирования.

```python
predictor = TabularPredictor(...).fit(
    train_dataset,
    time_limit=7200,  # 2 часа
    presets='best_quality',
    hyperparameters={...},
    num_bag_folds=5,
    num_stack_levels=2
)
```
*   **Обучение AutoGluon:**
    *   `time_limit`: Ограничение по времени обучения.
    *   `presets='best_quality'`: Настройка на максимальное качество, а не на скорость.
    *   `hyperparameters`: Можно тонко настраивать конкретные модели.
    *   `num_bag_folds=5`: **Бэггинг** — обучение нескольких копий модели на разных подвыборках.
    *   `num_stack_levels=2`: **Стекинг** — использование predictions первых-level моделей как features для meta-модели второго уровня. Мощный прием для увеличения точности.

```python
leaderboard = predictor.leaderboard()
```
*   **Leaderboard:** AutoGluon автоматически сравнивает все обученные модели и показывает их метрики. Позволяет увидеть, какая модель (или ансамбль) показала лучший результат.

**Ключевой вывод:** AutoGluon использован как высокоэффективный инструмент для автоматического построения сложного ансамбля моделей, избавляя исследователя от ручного подбора.



### **#7. Диагностика модели и анализ остатков**

**Цель:** Убедиться, что модель адекватна, и понять природу ее ошибок.

```python
# Анализ остатков
plt.scatter(y_pred, residuals, alpha=0.6)
stats.probplot(residuals, dist="norm", plot=plt)
plot_acf(residuals, lags=40)
```
*   **Диагностика остатков (Residual Analysis):**
    *   **Остатки vs Прогнозы:** Остатки должны быть случайно разбросаны вокруг нуля без каких-либо паттернов (например, в форме "воронки"). Паттерны означают, что модель не уловила какую-то закономерность.
    *   **Q-Q plot:** Проверка нормальности распределения остатков. Отклонение точек от прямой линии говорит о ненормальности.
    *   **Автокорреляция остатков:** Остатки не должны иметь значимой автокорреляции. Наличие автокорреляции (выходящие за синюю область столбцы) — признак того, что модель не использовала всю информацию из временных зависимостей.

```python
# Статистические тесты для остатков
shapiro_test = stats.shapiro(residuals)
```
*   **Тест Шапиро-Уилка:** Формальная проверка гипотезы о нормальности распределения остатков.

```python
# Анализ важности признаков
feature_importance = predictor.feature_importance(test_dataset)
```
*   **Важность признаков:** Показывает, какие признаки были наиболее важны для модели при принятии решений. Обычно это лаговые признаки и площадь.

```python
# SHAP значения
explainer = shap.Explainer(predictor.predict, sample_data)
shap_values = explainer(sample_data)
shap.summary_plot(shap_values.values, sample_data, plot_type="bar")
```
*   **SHAP (SHapley Additive exPlanations):** Передовой метод интерпретации моделей. Показывает не только важность, но и **направление влияния** каждого признака на прогноз (положительное или отрицательное).

**Ключевой вывод:** Проведена комплексная диагностика, которая показывает, насколько модель надежна, и помогает понять, как она принимает решения.



### **#8. Финальные результаты и выводы для статьи**

```python
# Финальные метрики
final_metrics = {'RMSE': ..., 'MAE': ..., 'R2': ..., 'MAPE': ...}
```
*   **Метрики качества:** Финальная оценка модели на тестовой выборке.
    *   **RMSE, MAE:** Ошибки в единицах целевой переменной (рубли). Показывают, "на сколько рублей в среднем ошибается модель".
    *   **R² (R-Squared):** Доля дисперсии, объясненная моделью. Ближе к 1 — лучше.
    *   **MAPE (Mean Absolute Percentage Error):** Средняя абсолютная процентная ошибка. Легко интерпретировать бизнесу.

```python
# Сравнение с baseline моделями
baseline_mae = mean_absolute_error(y_test, [y_train.mean()] * len(y_test))
print(f"Улучшение MAE: {(1 - final_metrics['MAE']/baseline_mae) * 100:.2f}%")
```
*   **Сравнение с Baseline:** Обязательный шаг. Baseline — это простейшая модель (например, прогноз всегда средним значением из тренировочного набора). Любая сложная модель должна быть лучше baseline. Процент улучшения — ключевой результат.

```python
# Генерация отчета
print("АНАЛИТИЧЕСКИЙ ОТЧЕТ")
...
```
*   **Подготовка к публикации:** Код генерирует сводный отчет и набор графиков, которые можно напрямую использовать в научной статье или отчете для заказчика.

**Ключевой вывод всей работы:** Представлен полный, воспроизводимый и хорошо документированный цикл анализа данных, который привел к созданию эффективной модели прогнозирования с измеримым улучшением качества относительно наивных методов. Код является отличным примером для изучения best practices в Data Science.